In [1]:
import os
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from scipy.special import expit

In [21]:
# Cargo el excel de la EPH
eph = pd.read_csv("eph_panel.csv")

C:\Users\felip\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (102,179,182,187,207,209) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
eph

,CODUSU,NRO_HOGAR,TRIMESTRE,ANO4,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,V19_B,IX_TOT,IX_MEN10,IX_MAYEQ10,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,1,1,42,N,26,165,...,2,2,0,2,2,0,98,0,0,0
1,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2016,1,1,42,N,26,176,...,2,2,0,2,2,0,98,0,0,0
2,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2017,1,1,42,N,26,159,...,2,2,0,2,2,0,98,0,0,0
3,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2017,1,1,42,N,26,171,...,2,2,0,2,2,1,98,0,0,0
4,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,2,1,42,N,26,165,...,2,2,0,2,2,0,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83115,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2017,1,1,44,N,31,141,...,2,2,0,2,97,0,98,0,0,0
83116,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2016,2,1,44,N,31,127,...,2,2,0,2,1,0,98,0,0,0
83117,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2016,2,1,44,N,31,148,...,2,2,0,2,1,0,2,0,0,0
83118,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2017,2,1,44,N,31,121,...,2,2,0,2,1,0,2,0,0,0


In [5]:
def limpiar_eph(eph):

    '''
        función para limpiar la eph
        input:
            eph: Variable que tenga almacenada la eph sin limpiar

        output:
            eph: Variable que tiene almacenada la eph limpia
    '''
    
    eph.dropna(thresh = 1, inplace = True) # Si tiene todos NA, la fila se va
    
    eph.dropna(thresh = 1, axis = "columns", inplace = True) # Si la columna tiene todos NA, la columna se va
    
    try:
        # Si no lo defino como uno me dropea un montón de filas que tienen NaN
        eph.PP08D1 = eph.PP08D1.fillna(0)
        eph.PP08F2 = eph.PP08F2.fillna(0)
        # dropeo observaciones con valores que no tienen sentido. CH06 = edad, PP08D1, P21, P47T, ITF y IPCF son todas variables
        # de ingresos, no pueden ser menores a 0
        eph.drop(eph[(eph["CH06"] < 0) | (eph["PP08D1"] < 0) | (eph["P21"] < 0) | \
                           (eph["P47T"] < 0) | (eph["ITF"] < 0) | (eph["IPCF"] < 0) | (eph["PP08F2"] < 0)].index, inplace = True)
        eph["CH14"].fillna(99, inplace = True) #Muchos no respondieron entonces reemplazamos los missing por 99.
        eph.fillna(0, inplace = True) # Relleno todos con 0 los missing de las columnas que representan subcategorías en
        #las que solo respondieron algunos. 
        #Para las siguientes variables dummys hay entre 15 y 20 observaciones para las que toma valor 9, es decir Ns/Nr.
        #Como son pocas observaciones y siempre las mismas, optamos por eliminarlas. 
        eph.drop(eph[(eph["V1"] == 9 ) | (eph["V2"] == 9) | (eph["V3"] == 9) | (eph["V4"] == 9) | (eph["V5"] == 9) | \
                      (eph["V6"] == 9 ) | (eph["V7"] == 9) | (eph["V8"] == 9) | (eph["V9"] == 9) | (eph["V10"] == 9) | \
                      (eph["V11"] == 9) | (eph["V12"] == 9) | (eph["V13"] == 9) | (eph["V14"] == 9) | (eph["V15"] == 9) | \
                      (eph["V16"] == 9) | (eph["V17"] == 9) | (eph["V18"] == 9) | (eph["V19_A"] == 9) | (eph["V19_B"] == 9)].index, inplace = True)
        eph.reset_index(inplace = True, drop = True)  
    except:
        print("No se encontró alguna columna, revisar")   

In [14]:
eph2 = eph.sort_values("CH06")

In [20]:
eph2[eph2["CH06"]==-9].CH06.describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: CH06, dtype: float64

In [ ]:
#Dejo como missing

In [6]:
limpiar_eph(eph)

In [7]:
eph

,CODUSU,NRO_HOGAR,TRIMESTRE,ANO4,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,V19_B,IX_TOT,IX_MEN10,IX_MAYEQ10,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,1,1,42,N,26,165,...,2,2,0,2,2,0,98,0,0,0
1,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2016,1,1,42,N,26,176,...,2,2,0,2,2,0,98,0,0,0
2,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2017,1,1,42,N,26,159,...,2,2,0,2,2,0,98,0,0,0
3,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2017,1,1,42,N,26,171,...,2,2,0,2,2,1,98,0,0,0
4,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,2,1,42,N,26,165,...,2,2,0,2,2,0,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75521,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2017,1,1,44,N,31,141,...,2,2,0,2,97,0,98,0,0,0
75522,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2016,2,1,44,N,31,127,...,2,2,0,2,1,0,98,0,0,0
75523,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2016,2,1,44,N,31,148,...,2,2,0,2,1,0,2,0,0,0
75524,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2017,2,1,44,N,31,121,...,2,2,0,2,1,0,2,0,0,0


In [22]:
#Importamos la tabla de excel
tabla_adulto = pd.read_excel("tabla_adulto_equiv.xlsx", header = 3)
#Ajustamos el formato de la tabla 
tabla_adulto.dropna(axis = 1, inplace = True, thresh = 10)
tabla_adulto.dropna(inplace = True, thresh = 3)
tabla_adulto.reset_index(inplace = True, drop = True)
#Creamos la columna "adulto_equiv" como una columna de ceros
eph["adulto_equiv"] = 0
#Generamos una nueva columna llamada lista como una columna de ceros 
eph["lista"] = 0
#Iteramos por filas
#Para cada fila del data frame de acuerdo a la edad le indicamos que 
#la celda de la columna "lista" la complete con el número correspondiente 
#al índice de la tabla_adulto. 
#Por ejemplo, si la edad es menor a 30 entonces se asigna el numero 18 que es
#el índice de la fila de 18 a 29 años. 
for i in eph.index:
    if eph.loc[i, "CH06"] < 19:
        eph.loc[i, "lista"] = eph.loc[i, "CH06"]
    elif eph.loc[i, "CH06"] < 30:
        eph.loc[i, "lista"] = 18
    elif eph.loc[i, "CH06"] < 46:
        eph.loc[i, "lista"] = 19
    elif eph.loc[i, "CH06"] < 61:
        eph.loc[i, "lista"] = 20
    elif eph.loc[i, "CH06"] < 76:
        eph.loc[i, "lista"] = 21
    else:
        eph.loc[i, "lista"] = 22
#CH06 = edad, CH04 = Sexo (1 = varon y 2 = mujer)
#Para cada fila del dataframe si el individuo es varón entonces le indicamos que bajo 
#la columna "adulto_equiv" coloque la siguiente celda de tabla_adulto: 
# Fila: el índice que esta guardado en la columna "lista" en el df eph. 
# Columna: Varones 
#idem para las mujeres
#De este modo se obtienen los valores de adulto equivalente de cada persona según 
#su sexo y edad
for i in eph.index:
    if eph.loc[i, "CH04"] == 1:
        eph.loc[i, "adulto_equiv"] = tabla_adulto.loc[eph.loc[i, "lista"], "Varones"]
    elif eph.loc[i, "CH04"] == 2:
        eph.loc[i, "adulto_equiv"] = tabla_adulto.loc[eph.loc[i, "lista"], "Mujeres"]
#Eliminamos la columna "lista"
eph.drop("lista", axis = 1, inplace = True)
#Realizamos un groupby: para cada hogar sumamos "adulto_equiv"
ad_equiv_hogar = eph[["CODUSU","ANO4","TRIMESTRE","adulto_equiv"]].groupby(["CODUSU","ANO4","TRIMESTRE"]).sum()
ad_equiv_hogar.reset_index(inplace = True)
#cambiamos los nombres de las columnas
ad_equiv_hogar.columns = ["CODUSU","ANO4","TRIMESTRE","ad_equiv_hogar"]
#agregamos la columnna "ad_equiv_hogar" al df eph
eph = eph.merge(ad_equiv_hogar, on = ["CODUSU","ANO4","TRIMESTRE"], how = "left")

KeyError: -1

In [ ]:
# Dividimos la muestra de la EPH en gente que no respondió ingresos totales familiares (ITF) y los que sí.
norespondieron = eph[eph["ITF"] == 0]
norespondieron.reset_index(inplace = True, drop = True)
respondieron = eph[eph["ITF"] != 0]
respondieron.reset_index(inplace = True, drop = True)

In [ ]:
# Ingresos mínimos por trimestre, hice un promedio del CBT por trimestre.
im_2t_2016 = 3812
im_3t_2016 = 4055
im_4t_2016 = 4232
im_1t_2017 = 4432
im_2t_2017 = 4744
im_3t_2017 = 4932
im_4t_2017 = 5219

# Creamos una columna de ingreso necesario para no ser pobre y la rellenamos con el for loop teniendo en cuenta el año
# y trimestre. Toma valores dependiendo de la columna ad_equiv_hogar * ingreso minimo per cápita.
respondieron["ingreso_necesario"] = 0
for i in respondieron.index:
    if (respondieron.loc[i, "ANO4"] == 2016) & (respondieron.loc[i, "TRIMESTRE"] == 2):
        respondieron.loc[i, "ingreso_necesario"] = im_2t_2016 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2016) & (respondieron.loc[i, "TRIMESTRE"] == 3):
        respondieron.loc[i, "ingreso_necesario"] = im_3t_2016 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2016) & (respondieron.loc[i, "TRIMESTRE"] == 4):
        respondieron.loc[i, "ingreso_necesario"] = im_4t_2016 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 1):
        respondieron.loc[i, "ingreso_necesario"] = im_1t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 2):
        respondieron.loc[i, "ingreso_necesario"] = im_2t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 3):
        respondieron.loc[i, "ingreso_necesario"] = im_3t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 4):
        respondieron.loc[i, "ingreso_necesario"] = im_4t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
        
# Creamos una columna dummy, que dice si el individuo es pobre o no. Depende de si sus ingresos
# son mayores a ingreso_necesario
respondieron["pobre"] = (respondieron["ITF"] < respondieron["ingreso_necesario"]).astype(int)

In [ ]:
respondieron.pobre.sum()

In [20]:
eph

,CODUSU,NRO_HOGAR,TRIMESTRE,ANO4,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,IX_MEN10,IX_MAYEQ10,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,adulto_equiv,ad_equiv_hogar
0,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,1,1,42,N,26,165,...,0,2,2,0,98,0,0,0,0.83,1.50
1,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2016,1,1,42,N,26,176,...,0,2,2,0,98,0,0,0,0.83,1.50
2,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2017,1,1,42,N,26,159,...,0,2,2,0,98,0,0,0,0.83,1.50
3,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2017,1,1,42,N,26,171,...,0,2,2,1,98,0,0,0,0.83,1.50
4,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,2,1,42,N,26,165,...,0,2,2,0,98,0,0,0,0.67,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75521,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2017,1,1,44,N,31,141,...,0,2,97,0,98,0,0,0,0.76,1.52
75522,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2016,2,1,44,N,31,127,...,0,2,1,0,98,0,0,0,0.76,1.52
75523,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2016,2,1,44,N,31,148,...,0,2,1,0,2,0,0,0,0.76,1.52
75524,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2017,2,1,44,N,31,121,...,0,2,1,0,2,0,0,0,0.76,1.52


In [21]:
respondieron

,CODUSU,NRO_HOGAR,TRIMESTRE,ANO4,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,adulto_equiv,ad_equiv_hogar,ingreso_necesario,pobre
0,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,1,1,42,N,26,165,...,2,0,98,0,0,0,0.83,1.50,5718.00,0
1,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2016,1,1,42,N,26,176,...,2,0,98,0,0,0,0.83,1.50,6082.50,0
2,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2017,1,1,42,N,26,159,...,2,0,98,0,0,0,0.83,1.50,7116.00,0
3,TQRMNOPPQHJKKTCDEHMHF00479193,1,3,2017,1,1,42,N,26,171,...,2,1,98,0,0,0,0.83,1.50,7398.00,0
4,TQRMNOPPQHJKKTCDEHMHF00479193,1,2,2016,2,1,42,N,26,165,...,2,0,98,0,0,0,0.67,1.50,5718.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64458,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2017,1,1,44,N,31,141,...,97,0,98,0,0,0,0.76,1.52,7932.88,0
64459,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2016,2,1,44,N,31,127,...,1,0,98,0,0,0,0.76,1.52,6163.60,0
64460,TQVMNOPXSHKNKPCDEIHJF00494445,1,4,2016,2,1,44,N,31,148,...,1,0,2,0,0,0,0.76,1.52,6432.64,0
64461,TQVMNOPXSHKNKPCDEIHJF00494445,1,3,2017,2,1,44,N,31,121,...,1,0,2,0,0,0,0.76,1.52,7496.64,0


In [ ]:
# Para arreglar que aparezcan si o si 4 veces, hago un count y si no aparecen 4 veces los elimino

In [25]:
arreglo = respondieron.groupby(["CODUSU","COMPONENTE"]).agg({"REGION":"count"})
arreglo[arreglo["REGION"]<4]

In [30]:
83120/4

20780.0

In [ ]:
# Preguntar que predecir, de que periodo a que.